<a href="https://colab.research.google.com/github/brucecmd/tensorflow_in_action_code/blob/master/simple_mlp_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

In [46]:
mnist = input_data.read_data_sets("./MNIST", one_hot=True)

Extracting ./MNIST/train-images-idx3-ubyte.gz
Extracting ./MNIST/train-labels-idx1-ubyte.gz
Extracting ./MNIST/t10k-images-idx3-ubyte.gz
Extracting ./MNIST/t10k-labels-idx1-ubyte.gz


In [47]:
print(mnist.train.num_examples)

55000


In [0]:
input_node = 784
output_node = 10

layer1_node = 500
batch_size = 100

learning_rate_base = 0.8
learning_rate_decay = 0.99

regularization_rate = 0.0001
traininig_steps = 30000
moving_average_decay = 0.99


In [0]:
def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
  if avg_class == None:
    layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
    return tf.matmul(layer1, weights2) + biases2
  else:
    layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
    return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)

In [0]:
def train(mnist):
  x = tf.placeholder(tf.float32, [None, input_node], name="x-input")
  y_ = tf.placeholder(tf.float32, [None, 10], name="y-input")
  
  # 用于训练的参数
  weights1 = tf.Variable(tf.truncated_normal([input_node, layer1_node], stddev=0.1))
  biases1 = tf.Variable(tf.constant(0.1, shape=[layer1_node]))
  weights2 = tf.Variable(tf.truncated_normal([layer1_node, output_node], stddev=0.1))
  biases2 = tf.Variable(tf.constant(0.1, shape=[output_node]))
  
  y = inference(x, None, weights1, biases1, weights2, biases2)
  
  # 准备滑动平均相关的东西
  global_step = tf.Variable(0, trainable=False)
  variable_averages = tf.train.ExponentialMovingAverage(moving_average_decay, global_step)
  variable_averages_op = variable_averages.apply(tf.trainable_variables()) # 注意，tf.trainable_variables()要加括号，不然报错
  
  average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
  
  cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_,1))
  cross_entropy_mean = tf.reduce_mean(cross_entropy)
  
  regularizer = tf.contrib.layers.l2_regularizer(regularization_rate)
  regularization = regularizer(weights1) + regularizer(weights2)
  loss = cross_entropy_mean + regularization
  
  learning_rate = tf.train.exponential_decay(learning_rate_base, global_step, mnist.train.num_examples/batch_size, learning_rate_decay)
  
  train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  with tf.control_dependencies([train_step, variable_averages_op]):
    train_op = tf.no_op(name='train')
    
  correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  
  with tf.Session() as sess:
    tf.global_variables_initializer().run()
    validate_feed = {x:mnist.validation.images,y_:mnist.validation.labels}
    test_feed = {x:mnist.test.images, y_:mnist.test.labels}
    
    for i in range(traininig_steps):
      if i % 1000 == 0:
        validate_acc = sess.run(accuracy, feed_dict=validate_feed)
        print("after %d steps, validation acc %g"%(i, validate_acc))
      xs,ys = mnist.train.next_batch(batch_size)
      sess.run(train_op, feed_dict={x:xs,y_:ys})
    test_acc = sess.run(accuracy, feed_dict=test_feed)
    print("after %d steps, acc is : %g"%(traininig_steps, test_acc))

In [55]:
train(mnist)

after 0 steps, validation acc 0.0962
after 1000 steps, validation acc 0.9764
after 2000 steps, validation acc 0.98
after 3000 steps, validation acc 0.983
after 4000 steps, validation acc 0.9828
after 5000 steps, validation acc 0.983
after 6000 steps, validation acc 0.9834
after 7000 steps, validation acc 0.9836
after 8000 steps, validation acc 0.9838
after 9000 steps, validation acc 0.9836
after 10000 steps, validation acc 0.9846
after 11000 steps, validation acc 0.9848
after 12000 steps, validation acc 0.9838
after 13000 steps, validation acc 0.9842
after 14000 steps, validation acc 0.9848
after 15000 steps, validation acc 0.984
after 16000 steps, validation acc 0.9848
after 17000 steps, validation acc 0.985
after 18000 steps, validation acc 0.9846
after 19000 steps, validation acc 0.9844
after 20000 steps, validation acc 0.9846
after 21000 steps, validation acc 0.9848
after 22000 steps, validation acc 0.9836
after 23000 steps, validation acc 0.9844
after 24000 steps, validation acc 0